# Composite Command

> aka Macro aka Composite Design Pattern

We will continue the scenario from the previous lesson. This time, we'll try to implement money transfers between 2 accounts.

We could try to create 2 commands, one for the withdrawal of money from one account and another for the deposit to the other account, but this approach is failure-prone. Before showing why, let's first import our old code:

In [1]:
# We import all of the previous code:
from abc import ABC, abstractmethod
from enum import Enum

class BankAccount:
    OVERDRAFT_LIMIT = -500

    def __init__(self, balance=0):
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount
        print(f'Deposited {amount}, balance = {self.balance}')

    def withdraw(self, amount):
        if self.balance - amount >= BankAccount.OVERDRAFT_LIMIT:
            self.balance -= amount
            print(f'Withdrew {amount}, balance = {self.balance}')
            return True
        return False

    def __str__(self):
        return f'Balance = {self.balance}'

class Command(ABC):
    def invoke(self):
        pass

    def undo(self):
        pass

class BankAccountCommand(Command):
    def __init__(self, account, action, amount):
        self.amount = amount
        self.action = action
        self.account = account
        self.success = None

    class Action(Enum):
        DEPOSIT = 0
        WITHDRAW = 1

    def invoke(self):
        if self.action == self.Action.DEPOSIT:
            self.account.deposit(self.amount)
            self.success = True
        elif self.action == self.Action.WITHDRAW:
            self.success = self.account.withdraw(self.amount)

    def undo(self):
        if not self.success:
            return
        if self.action == self.Action.DEPOSIT:
            self.account.withdraw(self.amount)
        elif self.action == self.Action.WITHDRAW:
            self.account.deposit(self.amount)

Before implementing the transfer, we'll try to perform 2 separate deposits on a bank account as a previous step to performing a transfer. We will run the commands one after another and see what happens. We want to perform both of these commands as a single command, so we'll create a special **composite command** that will consist of a list of commands:

In [2]:
class CompositeBankAccountCommand(Command, list):
    def __init__(self, items=[]):
        super().__init__()
        for i in items:
            self.append(i)

    def invoke(self):
        for x in self:
            x.invoke()

    def undo(self):
        for x in reversed(self): # we need to reverse our command list in order to undo all of its elements in order
            x.undo()

In [ ]:
# 2 separate deposits using a single composite command
ba = BankAccount()
deposit1 = BankAccountCommand(ba, BankAccountCommand.Action.DEPOSIT, 100)
deposit2 = BankAccountCommand(ba, BankAccountCommand.Action.DEPOSIT, 50)
composite = CompositeBankAccountCommand([deposit1, deposit2])

print(ba)
print('---')
composite.invoke()
print(ba)
print('---')
composite.undo()
print(ba)

Balance = 0
---
Deposited 100, balance = 100
Deposited 50, balance = 150
Balance = 150
---
Withdrew 50, balance = 100
Withdrew 100, balance = 0
Balance = 0


This code seems to be working fine. Now let's simulate a transfer between accounts with a composite command that will withdraw from one account and deposit into the other:

In [9]:
ba1 = BankAccount(100)
ba2 = BankAccount()

amount = 100
wc = BankAccountCommand(ba1, BankAccountCommand.Action.WITHDRAW, amount)
dc = BankAccountCommand(ba2, BankAccountCommand.Action.DEPOSIT, amount)

transfer = CompositeBankAccountCommand([wc, dc])

print('ba1:', ba1, 'ba2:', ba2)
print('---')
transfer.invoke()
print('ba1:', ba1, 'ba2:', ba2)
print('---')
transfer.undo()
print('ba1:', ba1, 'ba2:', ba2)

ba1: Balance = 100 ba2: Balance = 0
---
Withdrew 100, balance = 0
Deposited 100, balance = 100
ba1: Balance = 0 ba2: Balance = 100
---
Withdrew 100, balance = 0
Deposited 100, balance = 100
ba1: Balance = 100 ba2: Balance = 0


Once again, our code seems to be working fine, but actually it does not, because we can force invalid account balances by trying to transfer money that the accounts don't have. Let's see how:

In [10]:
ba1 = BankAccount(100)
ba2 = BankAccount()

amount = 1000  # we try to transfer more money than what's in the account
wc = BankAccountCommand(ba1, BankAccountCommand.Action.WITHDRAW, amount)
dc = BankAccountCommand(ba2, BankAccountCommand.Action.DEPOSIT, amount)

transfer = CompositeBankAccountCommand([wc, dc])

print('ba1:', ba1, 'ba2:', ba2)
print('---')
transfer.invoke()
print('ba1:', ba1, 'ba2:', ba2)  # end up in incorrect state
print('---')
transfer.undo()
print('ba1:', ba1, 'ba2:', ba2)

ba1: Balance = 100 ba2: Balance = 0
---
Deposited 1000, balance = 1000
ba1: Balance = 100 ba2: Balance = 1000
---
Withdrew 1000, balance = 0
ba1: Balance = 100 ba2: Balance = 0


If the first bank account only had $100, how is it possible that it transferred $1k to the second bank account? And why does the first bank account still have $100 after supposedly transferring that much money?

Our `BankAccountCommand` still has the success flag, but our `CompositeBankAccountCommand` isn't aware of them, and it so happened that in this last cell, the first action (`withdraw`) failed but the second action (`deposit`) was not aware of this, so it just went ahead and performed the action.

In order to fix this we need a way to implement a dependency between commands. We will move the success flag from `BankAccountCommand` to the `Command` base class modify `BankAccountCommand` to call its superclass:

In [12]:
class Command(ABC):
    def __init__(self):
        self.success = False # we assume that a command always fails unless we explicitly say otherwise
    def invoke(self):
        pass
    def undo(self):
        pass

class BankAccountCommand(Command):
    def __init__(self, account, action, amount):
        super().__init__() # We make sure that we initialize the flag from the superclass
        self.amount = amount
        self.action = action
        self.account = account
    class Action(Enum):
        DEPOSIT = 0
        WITHDRAW = 1
    def invoke(self):
        if self.action == self.Action.DEPOSIT:
            self.account.deposit(self.amount)
            self.success = True
        elif self.action == self.Action.WITHDRAW:
            self.success = self.account.withdraw(self.amount)
    def undo(self):
        if not self.success:
            return
        if self.action == self.Action.DEPOSIT:
            self.account.withdraw(self.amount)
        elif self.action == self.Action.WITHDRAW:
            self.account.deposit(self.amount)

# We don't need to do any changes to this class, but we need to run the code again in order to have it work as expected
class CompositeBankAccountCommand(Command, list):
    def __init__(self, items=[]):
        super().__init__()
        for i in items:
            self.append(i)
    def invoke(self):
        for x in self:
            x.invoke()
    def undo(self):
        for x in reversed(self):
            x.undo()

Since `CompositeBankAccountCommand` inherits from `Command` and it already calls `super().__init__()` when initializing, we don't need to make any modifications to it.

With all of this in place, we're just missing the last part: managing the flags from our composite command. We'll create a new specific `MoneyTransferCommand` class that will inherit from `CompositeBankAccountCommand` and will take care of flag management:

In [18]:
class MoneyTransferCommand(CompositeBankAccountCommand):
    def __init__(self, from_acct, to_acct, amount):
        """We call super.init with a list of a withdrawal and deposit with the amount we want"""
        super().__init__([
            BankAccountCommand(from_acct, BankAccountCommand.Action.WITHDRAW, amount),
            BankAccountCommand(to_acct, BankAccountCommand.Action.DEPOSIT, amount)
        ])
        
    def invoke(self):
        ok = True # flag to determine whether the previous action actually succeeded
        for cmd in self:
            if ok:
                cmd.invoke()
                ok = cmd.success
            else:
                cmd.success = False
        self.success = ok # we store whether or not the full composite command succeeded

This composite command will make sure that if one of the commands fails, all subsequent commands in the list will be flagged as failed as well.

We can now test our code and see how it works by recreating our previous 2 transfer examples:

In [19]:
# valid transfer
ba1 = BankAccount(100)
ba2 = BankAccount()

amount = 100

transfer = MoneyTransferCommand(ba1, ba2, amount)

print('ba1:', ba1, 'ba2:', ba2)
print('---')
transfer.invoke()
print('ba1:', ba1, 'ba2:', ba2)
print('---')
transfer.undo()
print('ba1:', ba1, 'ba2:', ba2)
print(transfer.success)

ba1: Balance = 100 ba2: Balance = 0
---
Withdrew 100, balance = 0
Deposited 100, balance = 100
ba1: Balance = 0 ba2: Balance = 100
---
Withdrew 100, balance = 0
Deposited 100, balance = 100
ba1: Balance = 100 ba2: Balance = 0
True


In [20]:
# illegal transfer
ba1 = BankAccount(100)
ba2 = BankAccount()

amount = 1000

transfer = MoneyTransferCommand(ba1, ba2, amount)

print('ba1:', ba1, 'ba2:', ba2)
print('---')
transfer.invoke()
print('ba1:', ba1, 'ba2:', ba2)
print('---')
transfer.undo()
print('ba1:', ba1, 'ba2:', ba2)
print(transfer.success)

ba1: Balance = 100 ba2: Balance = 0
---
ba1: Balance = 100 ba2: Balance = 0
---
ba1: Balance = 100 ba2: Balance = 0
False


And that's how a composite command works: by creating a list of commands and providing the additional means for controlling each step of the chain.